In [15]:
import math
import numpy as np

class FourThrustSimulator:
    _instance = None
    
    def __init__(self, initial_pitch=0, initial_roll=0, initial_yaw=0, speed=1):
        # Physical constants
        self.mass = 10.0  # kg
        self.mass *= 2.20462
        self.arm_length = 0.2  # meters (20cm)
        self.gravity = 9.81  # m/s^2
        self.dt = 0.01  # seconds
        self.speed = speed
        self.time = 0
        
        # Moments of inertia (kg*m^2)
        self.I_xx = self.mass * (self.arm_length ** 2) / 2  # Roll
        self.I_yy = self.I_xx  # Pitch
        self.I_zz = self.mass * (self.arm_length ** 2)  # Yaw
        
        # State variables (in radians)
        self.pitch = math.radians(initial_pitch)
        self.roll = math.radians(initial_roll)
        self.yaw = math.radians(initial_yaw)
        
        # Angular velocities (rad/s)
        self.pitch_rate = 0.0
        self.roll_rate = 0.0
        self.yaw_rate = 0.0
        
        # Base damping coefficients
        self.base_damping_pitch = 0.05
        self.base_damping_roll = 0.05
        self.base_damping_yaw = 0.15

        # Damping growth factors
        self.damping_growth_rate = 0.1  # How quickly damping increases
        self.max_damping = 0.5         # Maximum damping coefficient

        # Initialize current damping (will increase over time)
        self.damping_pitch = self.base_damping_pitch
        self.damping_roll = self.base_damping_roll
        self.damping_yaw = self.base_damping_yaw

        # Energy loss tracking
        self.time = 0
        self.last_energy = None

        # Thrust values (N)
        self.thrusts = {
            'front_left': 0,
            'front_right': 0,
            'back_left': 0,
            'back_right': 0
        }

        # Thruster positions relative to COM (meters)
        self.thruster_positions = {
            'front_left': (-self.arm_length/2, self.arm_length/2, 0),
            'front_right': (-self.arm_length/2, -self.arm_length/2, 0),
            'back_left': (self.arm_length/2, self.arm_length/2, 0),
            'back_right': (self.arm_length/2, -self.arm_length/2, 0)
        }

    def psi_to_newtons(self, psi):
        """Convert PSI to Newtons (approximate conversion)"""
        # 1 PSI ≈ 6.89476 kPa
        # Force = Pressure * Area (assuming 1 square inch nozzle)
        # 1 square inch = 0.00064516 square meters
        return psi * 6.89476 * 0.00064516 * 1000  # Convert to Newtons

    def calculate_torques(self):
        """Calculate torques using proper cross products and moment arms"""
        torques = np.zeros(3)
        
        for thruster, position in self.thruster_positions.items():
            force = np.array([0, 0, self.thrusts[thruster]])
            pos = np.array(position)
            torque = np.cross(pos, force)
            torques += torque
            
        return torques

    def apply_thrust(self, front_left_psi=0, front_right_psi=0, back_left_psi=0, back_right_psi=0):
        """Apply thrust forces and update the system state"""
        # Convert PSI to Newtons
        self.thrusts['front_left'] = self.psi_to_newtons(front_left_psi)
        self.thrusts['front_right'] = self.psi_to_newtons(front_right_psi)
        self.thrusts['back_left'] = self.psi_to_newtons(back_left_psi)
        self.thrusts['back_right'] = self.psi_to_newtons(back_right_psi)
        
        # Calculate torques
        torques = self.calculate_torques()
        
        # Calculate angular accelerations using proper moments of inertia
        pitch_acc = torques[0] / self.I_yy
        roll_acc = torques[1] / self.I_xx
        yaw_acc = torques[2] / self.I_zz
        
        # Update angular velocities
        self.pitch_rate += pitch_acc * self.dt * self.speed
        self.roll_rate += roll_acc * self.dt * self.speed
        self.yaw_rate += yaw_acc * self.dt * self.speed

    def update(self):
        """Update drone state using physics-based equations of motion with increasing energy loss"""
        # Calculate gravitational torques (pendulum effect)
        g_torque_pitch = -(self.mass * self.gravity * self.arm_length) * math.sin(self.pitch)
        g_torque_roll = -(self.mass * self.gravity * self.arm_length) * math.sin(self.roll)

        # Calculate angular accelerations from gravitational torques
        pitch_acc = g_torque_pitch / self.I_yy
        roll_acc = g_torque_roll / self.I_xx

        # Update damping based on time and motion
        # Damping increases with time but is capped at max_damping
        time_factor = min(self.time / 10.0, 1.0)  # Reaches max effect after 10 seconds

        # Calculate current energy (kinetic + potential)
        current_energy = (
            0.5 * (self.I_yy * self.pitch_rate**2 + self.I_xx * self.roll_rate**2) +  # Kinetic
            self.mass * self.gravity * self.arm_length * (1 - math.cos(self.pitch))    # Potential
        )

        # Increase damping based on both time and energy
        energy_factor = 1.0
        if self.last_energy is not None:
            # Increase damping more when energy is higher
            energy_factor = min(current_energy / self.last_energy, 1.5) if self.last_energy > 0 else 1.0

        self.last_energy = current_energy

        # Calculate new damping coefficients
        self.damping_pitch = min(
            self.base_damping_pitch * (1 + self.damping_growth_rate * time_factor * energy_factor),
            self.max_damping
        )
        self.damping_roll = min(
            self.base_damping_roll * (1 + self.damping_growth_rate * time_factor * energy_factor),
            self.max_damping
        )

        # Add damping effects (increasing with time)
        self.pitch_rate -= self.damping_pitch * self.pitch_rate
        self.roll_rate -= self.damping_roll * self.roll_rate
        self.yaw_rate -= self.damping_yaw * self.yaw_rate

        # Update angular velocities using accelerations
        self.pitch_rate += pitch_acc * self.dt * self.speed
        self.roll_rate += roll_acc * self.dt * self.speed

        # Update angles using angular velocities
        self.pitch += self.pitch_rate * self.dt * self.speed
        self.roll += self.roll_rate * self.dt * self.speed
        self.yaw += self.yaw_rate * self.dt * self.speed

        # Constrain angles to prevent unrealistic behavior
        self.pitch = max(min(self.pitch, math.pi/2), -math.pi/2)
        self.roll = max(min(self.roll, math.pi/2), -math.pi/2)
        self.yaw = self.yaw % (2 * math.pi)

        self.time += self.dt

    def is_stable(self, target_pitch=0, target_roll=0, target_yaw=0):
        """Check if the drone is stable within specified tolerances"""
        angle_tolerance = math.radians(0.5)  # 0.5 degrees
        rate_tolerance = 0.1  # rad/s
        
        return (
            abs(self.pitch - math.radians(target_pitch)) < angle_tolerance and
            abs(self.roll - math.radians(target_roll)) < angle_tolerance and
            abs(self.yaw - math.radians(target_yaw)) < angle_tolerance and
            abs(self.pitch_rate) < rate_tolerance and
            abs(self.roll_rate) < rate_tolerance and
            abs(self.yaw_rate) < rate_tolerance
        )

    def get_ypr(self):
        """Get Yaw, Pitch, Roll values in degrees"""
        return [
            math.degrees(self.yaw),
            math.degrees(self.pitch),
            math.degrees(self.roll)
        ]
    
    def get_ypr_rates(self):
        """Get Yaw, Pitch, Roll values in degrees"""
        return [
            math.degrees(self.yaw_rate),
            math.degrees(self.pitch_rate),
            math.degrees(self.roll_rate)
        ]
    
    def get_motion_data(self):
        """Get detailed motion data in degrees and deg/s"""
        return {
            'pitch': math.degrees(self.pitch),
            'pitch_rate': math.degrees(self.pitch_rate),
            'roll': math.degrees(self.roll),
            'roll_rate': math.degrees(self.roll_rate),
            'yaw': math.degrees(self.yaw),
            'yaw_rate': math.degrees(self.yaw_rate),
            'is_stable': self.is_stable()
        }

    def reset(self, initial_pitch=0, initial_roll=0, initial_yaw=0, speed=1):
        """Reset the simulator to initial conditions"""
        self.__init__(initial_pitch, initial_roll, initial_yaw, speed)
        self.time = 0
        self.last_energy = None

    @classmethod
    def get_instance(cls, initial_angle=0, speed=1):
        if cls._instance is None:
            cls._instance = cls(initial_angle, speed)
        return cls._instance

def get_simulated_angle(max_steps=300):
    simulator = FourThrustSimulator.get_instance()
    if not simulator.is_stable() and max_steps > 0:
        simulator.update()
    return simulator.get_motion_data()

In [23]:
# Set the thrust PSI value and target orientations
PSI = 60  # Thrst pressure in PSI
TARGET_PITCH = 10  # Target pitch in degrees
TARGET_ROLL = 20   # Target roll in degrees
THRESHOLD = 5     # Threshold for minimal pitch and roll adjustments

def no_opps(front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust):
    """
    Checks and disables opposing thruster pairs if both are active.
    
    Args:
        front_left_thrust (float): Front left thruster value
        front_right_thrust (float): Front right thruster value
        back_left_thrust (float): Back left thruster value
        back_right_thrust (float): Back right thruster value
        
    Returns:
        tuple: Updated thrust values with opposing pairs disabled
    """
    # Check diagonal pairs (front-left vs back-right)
    if front_left_thrust > 0 and back_right_thrust > 0:
        front_left_thrust = back_right_thrust = 0
        
    # Check diagonal pairs (front-right vs back-left)
    if front_right_thrust > 0 and back_left_thrust > 0:
        front_right_thrust = back_left_thrust = 0
        
    return front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust

def thrust_control(current_pitch: float, current_roll: float):
    """
    Controls thrusters to achieve target pitch and roll angles.
    
    Thrust effects:
    - Two back thrusters ON: Roll decreases
    - Two front thrusters ON: Roll increases
    - Two right thrusters ON: Pitch decreases
    - Two left thrusters ON: Pitch increases
    
    Args:
        current_pitch (float): Current pitch angle in degrees
        current_roll (float): Current roll angle in degrees
        
    Returns:
        tuple: Calculated thrust values for all four thrusters
    """
    # Initialize all thrusters to zero
    front_left_thrust = front_right_thrust = back_left_thrust = back_right_thrust = 0
    
    # Calculate errors
    pitch_error = TARGET_PITCH - current_pitch
    roll_error = TARGET_ROLL - current_roll
    
    # If both pitch and roll errors are below the threshold, set all thrusts to zero
    if abs(pitch_error) < THRESHOLD and abs(roll_error) < THRESHOLD:
        print("Pitch and Roll errors are within threshold. No thrust applied.")
        return 0, 0, 0, 0
    
    print(f"Pitch Error: {pitch_error}")
    print(f"Roll Error: {roll_error}")
    
    # Pitch correction
    if pitch_error > 0:  # Need to increase pitch - activate left thrusters
        front_left_thrust = PSI
        back_left_thrust = PSI
    elif pitch_error < 0:  # Need to decrease pitch - activate right thrusters
        front_right_thrust = PSI
        back_right_thrust = PSI
        
    # Roll correction
    if roll_error > 0:  # Need to increase roll - activate front thrusters
        front_left_thrust = PSI
        front_right_thrust = PSI
    elif roll_error < 0:  # Need to decrease roll - activate back thrusters
        back_left_thrust = PSI
        back_right_thrust = PSI
    
    # Prevent opposing thrusters from firing simultaneously
    thrusts = no_opps(
        front_left_thrust,
        front_right_thrust,
        back_left_thrust,
        back_right_thrust
    )
    
    # Print final thrust values
    print(f"Thrusts (FL, FR, BL, BR): {thrusts}")
    
    return thrusts

Pitch and Roll errors are within threshold. No thrust applied.


In [19]:
# import time

# # Target pitch and roll values
# target_pitch = 0
# target_roll = 0

# # Run the simulation to debug the correction logic
# def run_simulation(current_pitch, current_roll, steps=300):
#     print(f"Starting simulation with initial pitch: {current_pitch}° and roll: {current_roll}°")
#     print("Step | Pitch Error | Roll Error | Target Pitch | Target Roll | Thrusts (FL, FR, BL, BR) | Current Pitch | Current Roll | Yaw")
#     print("-" * 120)
    
#     # Initialize the FourThrustSimulator with the initial pitch and roll
#     simulator = FourThrustSimulator.get_instance(speed=10)

#     for step in range(steps):
#         # Get the current pitch, roll, and yaw from the simulator
#         current_pitch, current_roll, current_yaw = simulator.get_ypr()

#         # Calculate thrusts to bring pitch and roll toward targets
#         front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust = thrust_control(current_pitch, current_roll)
        
#         # Print the current state, errors, and thrust values for debugging
#         print(f"{step + 1:4d} | "
#               f"{target_pitch - current_pitch:+8.2f} | {target_roll - current_roll:+8.2f} | "
#               f"{target_pitch:+8.2f} | {target_roll:+8.2f} | "
#               f"{front_left_thrust:3d}, {front_right_thrust:3d}, {back_left_thrust:3d}, {back_right_thrust:3d} | "
#               f"{current_pitch:+8.2f} | {current_roll:+8.2f} | {current_yaw:+8.2f}")
        

#         # Apply thrusts to the simulator
#         simulator.apply_thrust(front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust)

#         # Update the simulator's state to apply physics and thrusts
#         simulator.update()
#         print(simulator.get_ypr())
        
#         # Check if the simulator has stabilized
#         if simulator.is_stable(simulator.get_ypr()[0],simulator.get_ypr()[1],simulator.get_ypr()[2]):
#             print("\nisStable!")
#             break
        
#         time.sleep(0.1)  # Small delay for readability (adjust or remove as needed)
    
#     print("Simulation complete.")

# # Usage example
# if __name__ == "__main__":
#     current_pitch = float(input("Enter current pitch (-50 to 50 degrees): "))
#     current_roll = float(input("Enter current roll (-50 to 50 degrees): "))
    
#     if -50 <= current_pitch <= 50 and -50 <= current_roll <= 50:
#         run_simulation(current_pitch, current_roll)
#     else:
#         print("Please enter valid pitch and roll values between -50 and 50 degrees.")

In [21]:
import numpy as np
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go

# Initialize the app
app = dash.Dash(__name__)
graphid = "drone-graph"

# Create a layout with a graph, interval, and controls
app.layout = html.Div([
    html.Div([
        html.Div(id='ypr-display', style={'fontSize': 18, 'margin': '10px'}),
    ]),
    html.Div([
        html.Button('Reset', id='reset-button', n_clicks=0),
        dcc.Input(
            id='initial-pitch',
            type='number',
            min=-90,
            max=90,
            placeholder='Initial Pitch Angle (-90 to 90)',
            value=10,
            style={'margin': '10px'}
        ),
        dcc.Input(
            id='initial-roll',
            type='number',
            min=-90,
            max=90,
            placeholder='Initial Roll Angle (-90 to 90)',
            value=0,
            style={'margin': '10px'}
        ),
        dcc.Input(
            id='initial-yaw',
            type='number',
            min=-180,
            max=180,
            placeholder='Initial Yaw Angle (-180 to 180)',
            value=0,
            style={'margin': '10px'}
        ),
        html.Div([
            html.Label('Front_L Thruster (PSI):'),
            dcc.Input(
                id='front-left-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'width': '50px', 'margin': '10px'}
            ),
            html.Label('Front_R Thruster (PSI):'),
            dcc.Input(
                id='front-right-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'width': '50px', 'margin': '10px'}
            ),
            html.Label('Back_L Thruster (PSI):'),
            dcc.Input(
                id='back-left-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'width': '50px', 'margin': '10px'}
            ),
            html.Label('Back_R Thruster (PSI):'),
            dcc.Input(
                id='back-right-thrust',
                type='number',
                min=0,
                max=100,
                value=0,
                style={'width': '50px', 'margin': '10px'}
            ),
            html.Button('Orbit Left', id='orbit-left', n_clicks=0),
            html.Button('Orbit Right', id='orbit-right', n_clicks=0),
            html.Button('Orbit Up', id='orbit-up', n_clicks=0),
            html.Button('Orbit Down', id='orbit-down', n_clicks=0),
        ]),
    ], style={'margin': '10px'}),

    dcc.Store(id='camera-store', data={'azimuth': 25, 'elevation': 30, 'distance': 2}),
    dcc.Graph(id=graphid),
    dcc.Interval(
        id='update-interval',
        interval=20, # Xms speed
        n_intervals=0
    )
])

# Initialize the simulator instance
simulator = FourThrustSimulator.get_instance()

def create_rotation_matrix(yaw, pitch, roll):
    """Create a proper rotation matrix using ZYX (yaw, pitch, roll) convention"""
    # Convert angles to radians if they aren't already
    yaw = np.radians(yaw)
    pitch = np.radians(pitch)
    roll = np.radians(roll)
    
    # Individual rotation matrices
    Rz = np.array([
        [np.cos(yaw), -np.sin(yaw), 0],
        [np.sin(yaw), np.cos(yaw), 0],
        [0, 0, 1]
    ])
    
    Ry = np.array([
        [np.cos(pitch), 0, np.sin(pitch)],
        [0, 1, 0],
        [-np.sin(pitch), 0, np.cos(pitch)]
    ])
    
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(roll), -np.sin(roll)],
        [0, np.sin(roll), np.cos(roll)]
    ])
    
    # Combine rotations in ZYX order (yaw, pitch, roll)
    R = Rz @ Ry @ Rx
    return R

@app.callback(
    [Output(graphid, 'figure'), Output('camera-store', 'data'), Output('ypr-display', 'children')],
    [Input('update-interval', 'n_intervals'), 
     Input('front-left-thrust', 'value'), 
     Input('front-right-thrust', 'value'),
     Input('back-left-thrust', 'value'), 
     Input('back-right-thrust', 'value'), 
     Input('orbit-left', 'n_clicks'),
     Input('orbit-right', 'n_clicks'), 
     Input('orbit-up', 'n_clicks'), 
     Input('orbit-down', 'n_clicks')],
    [State('camera-store', 'data')]
)
def update_drone_graph(n, front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust,
                      orbit_left, orbit_right, orbit_up, orbit_down, camera_data):
    
    if front_left_thrust != 0 or front_right_thrust != 0 or back_left_thrust != 0 or back_right_thrust != 0:
        # Apply thrust to the simulator
        simulator.apply_thrust(front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust)
    else:
        yaw, pitch, roll = simulator.get_ypr()
        yaw_rate, pitch_rate, roll_rate = simulator.get_ypr_rates()
        front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust = thrust_control(pitch, roll)
        simulator.apply_thrust(front_left_thrust, front_right_thrust, back_left_thrust, back_right_thrust)
    
    simulator.update()

    # Retrieve yaw, pitch, and roll from the simulator
    yaw, pitch, roll = simulator.get_ypr()
    # Update the YPR display text
    ypr_text = f"Pitch: {pitch:.2f}°, Roll: {roll:.2f}°, Yaw: {yaw:.2f}°"
    
    # Define the drone frame vertices (scaled down from previous version)
    length = 0.5  # Smaller size for better visualization
    vertices = np.array([
        [-length, length, 0],   # Front Left
        [length, length, 0],    # Front Right
        [-length, -length, 0],  # Back Left
        [length, -length, 0]    # Back Right
    ])
    
    # Create rotation matrix and apply to vertices
    R = create_rotation_matrix(yaw, pitch, roll)
    rotated_vertices = vertices @ R.T
    
    # Extract coordinates
    x_values = rotated_vertices[:, 0]
    y_values = rotated_vertices[:, 1]
    z_values = rotated_vertices[:, 2]
    
    # Scale the cross size relative to the drone frame
    cross_length = length/2  # Using the same length variable defined earlier

    # Define cross vertices
    cross_vertices = np.array([
        # Z-axis (vertical)
        [0, 0, -cross_length],
        [0, 0, cross_length],
        # X-axis (front-back)
        [-cross_length, 0, 0],
        [cross_length, 0, 0],
        # Y-axis (left-right)
        [0, -cross_length, 0],
        [0, cross_length, 0]
    ])

    # Apply the same rotation matrix to cross vertices
    rotated_cross = cross_vertices @ R.T
    
    # Create the visualization traces
    traces = [
        # Drone frame
        go.Scatter3d(
            x=np.append(x_values, x_values[0]),
            y=np.append(y_values, y_values[0]),
            z=np.append(z_values, z_values[0]),
            mode='lines',
            line=dict(color='blue', width=4),
            name='Drone Frame'
        ),
        # Motors
        go.Scatter3d(
            x=x_values,
            y=y_values,
            z=z_values,
            mode='markers+text',
            marker=dict(size=8, color='orange', symbol='circle'),
            text=[f'FL: {front_left_thrust}', f'FR: {front_right_thrust}',
                  f'BL: {back_left_thrust}', f'BR: {back_right_thrust}'],
            textposition='bottom center',
            name='Motors'
        ),
        # Center cross
        # Z axis (vertical)
        go.Scatter3d(
            x=rotated_cross[0:2, 0],
            y=rotated_cross[0:2, 1],
            z=rotated_cross[0:2, 2],
            mode='lines',
            line=dict(color='red', width=2),
            name='Z axis'
        ),
        # X axis (front-back)
        go.Scatter3d(
            x=rotated_cross[2:4, 0],
            y=rotated_cross[2:4, 1],
            z=rotated_cross[2:4, 2],
            mode='lines',
            line=dict(color='green', width=2),
            name='X axis'
        ),
        # Y axis (left-right)
        go.Scatter3d(
            x=rotated_cross[4:6, 0],
            y=rotated_cross[4:6, 1],
            z=rotated_cross[4:6, 2],
            mode='lines',
            line=dict(color='blue', width=2),
            name='Y axis'
        )
    ]

    # Update camera position
    camera_data['azimuth'] += (orbit_right - orbit_left) * 2
    camera_data['elevation'] += (orbit_up - orbit_down) * 2
    
    # Create the layout
    layout = go.Layout(
        scene=dict(
            xaxis=dict(range=[-1, 1], showgrid=True, zeroline=True),
            yaxis=dict(range=[-1, 1], showgrid=True, zeroline=True),
            zaxis=dict(range=[-1, 1], showgrid=True, zeroline=True),
            camera=dict(
                eye=dict(
                    x=camera_data['distance'] * np.cos(np.radians(camera_data['azimuth'])) * 
                      np.cos(np.radians(camera_data['elevation'])),
                    y=camera_data['distance'] * np.sin(np.radians(camera_data['azimuth'])) * 
                      np.cos(np.radians(camera_data['elevation'])),
                    z=camera_data['distance'] * np.sin(np.radians(camera_data['elevation']))
                )
            ),
            aspectmode='cube'
        ),
        title='Drone Simulation',
        showlegend=False,
        margin=dict(l=0, r=0, t=40, b=0)
    )
    
    return {'data': traces, 'layout': layout}, camera_data, ypr_text

@app.callback(
    Output('update-interval', 'n_intervals'),
    Input('reset-button', 'n_clicks'),
    [State('initial-pitch', 'value'),
     State('initial-roll', 'value'),
     State('initial-yaw', 'value')]
)
def reset_simulation(n_clicks, initial_pitch, initial_roll, initial_yaw):
    if n_clicks > 0:
        simulator.reset(initial_pitch=initial_pitch, initial_roll=initial_roll, initial_yaw=initial_yaw)
    return 0

if __name__ == '__main__':
    app.run_server(debug=True)

In [10]:
# 